# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,69.01,86,5,3.00,AU,1613436465
1,1,kaeo,-35.1000,173.7833,77.00,80,82,15.99,NZ,1613436610
2,2,muroto,33.2833,134.1500,48.20,49,40,2.30,JP,1613436635
3,3,ushuaia,-54.8000,-68.3000,57.20,51,75,13.80,AR,1613436352
4,4,rikitea,-23.1203,-134.9692,80.19,77,0,12.01,PF,1613436635


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,arraial do cabo,-22.9661,-42.0278,77.43,87,0,7.29,BR,1613436479
25,25,new norfolk,-42.7826,147.0587,73.00,53,0,1.01,AU,1613436480
148,148,sur,22.5667,59.5289,70.66,57,0,4.52,OM,1613436647
169,169,saint-pierre,-21.3393,55.4781,78.80,69,0,6.91,RE,1613436649
281,281,saint-joseph,-21.3667,55.6167,78.80,69,0,6.91,RE,1613436658
322,322,saint-philippe,-21.3585,55.7679,78.80,69,0,6.91,RE,1613436662
364,364,kontagora,10.3999,5.4695,77.45,16,0,6.08,NG,1613436666
404,404,si sa ket,14.8333,104.3333,71.01,67,0,8.08,TH,1613436669
476,476,ranong,9.9658,98.6348,71.60,94,0,3.51,TH,1613436675
494,494,mollendo,-17.0231,-72.0147,71.17,77,0,7.58,PE,1613436677


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
5,arraial do cabo,BR,-22.9661,-42.0278,
25,new norfolk,AU,-42.7826,147.0587,
148,sur,OM,22.5667,59.5289,
169,saint-pierre,RE,-21.3393,55.4781,
281,saint-joseph,RE,-21.3667,55.6167,
322,saint-philippe,RE,-21.3585,55.7679,
364,kontagora,NG,10.3999,5.4695,
404,si sa ket,TH,14.8333,104.3333,
476,ranong,TH,9.9658,98.6348,
494,mollendo,PE,-17.0231,-72.0147,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df


,City,Country,Lat,Lng,Hotel Name
5,arraial do cabo,BR,-22.9661,-42.0278,Pousada Porto Praia
25,new norfolk,AU,-42.7826,147.0587,The Shingles Riverside Cottages
148,sur,OM,22.5667,59.5289,Sur Plaza Hotel
169,saint-pierre,RE,-21.3393,55.4781,Lindsey Hôtel
281,saint-joseph,RE,-21.3667,55.6167,"""Plantation Bed and Breakfast"
322,saint-philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
364,kontagora,NG,10.3999,5.4695,City Gate Inn
404,si sa ket,TH,14.8333,104.3333,ร้านค้า อัญชัน
476,ranong,TH,9.9658,98.6348,Tinidee Hotel@Ranong
494,mollendo,PE,-17.0231,-72.0147,Hostal La Casa de Melgar


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))